In [4]:
import pandas as pd

In [5]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data (1).csv


In [6]:
df = pd.read_csv("data.csv")
df

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1
...,...,...,...,...
4004,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0
4005,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0
4006,https://www.activistpost.com/2017/09/ron-paul-...,"Ron Paul on Trump, Anarchism & the AltRight",NaN,0
4007,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1


In [7]:
print(df.info())
print('-------------------------------------------------------')
print(df.isna().sum())
print('-------------------------------------------------------')
print(df['Label'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URLs      4009 non-null   object
 1   Headline  4009 non-null   object
 2   Body      3988 non-null   object
 3   Label     4009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 125.4+ KB
None
-------------------------------------------------------
URLs         0
Headline     0
Body        21
Label        0
dtype: int64
-------------------------------------------------------
0    2137
1    1872
Name: Label, dtype: int64


# Обработка текста



In [8]:
df = df.dropna().reset_index(drop = True)
print(df.isna().sum())

URLs        0
Headline    0
Body        0
Label       0
dtype: int64


In [9]:
from string import punctuation
import re

def remove_punct(text):
    # удаление пунктуации в тексте
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ',
             43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ',
             63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def txt_prep(df):
    # функция приводит весь текст к нижнему регистру
    # удаляет пунктуацию
    df['Body_01'] = df['Body']
    df['Body_01'] = df['Body_01'].str.lower()
    # остаётся вопрос - нужно ли убирать 's или 'll - эти сокращения явно признак фейковости,
    # после чистки пунктуации там останется s - скорее всего проверка пунктуации определит это как ошибку
    df['Body_01'] = df['Body_01'].map(lambda x: remove_punct(x))

    signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', '@', '>', '<', '–', '&', '%', '•', '/', '“']
    for punct_sign in signs:
        df['Body_01'] = df['Body_01'].str.replace(punct_sign, ' ')

    df['Body_01'] = df['Body_01'].str.replace('\n', '')
    df['Body_01'] = df['Body_01'].str.replace(r"\d+", "", flags=re.UNICODE)
    df['Body_01'] = df['Body_01'].str.replace("  " , " ")
    df['Body_01'] = df['Body_01'].str.replace("  " , " ")

    return df

df = txt_prep(df)

In [10]:
df.head()

,URLs,Headline,Body,Label,Body_01
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...


## Проверка и исправление на ошибки

In [11]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [12]:
!sudo pip install jamspell

In [13]:
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz

--2021-12-02 22:15:25--  https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz [following]
--2021-12-02 22:15:25--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36611828 (35M) [application/octet-stream]
Saving to: ‘en.tar.gz.1’

en.tar.gz.1         100%[===================>]  34.92M  91.0MB/s    in 0.4s    

2021-12-02 22:15:26 (91.0 MB/s) - ‘en.tar.gz.1’ saved [36611828/36611828]



In [14]:
!tar -xvf en.tar.gz

en.bin


In [15]:
from jamspell import TSpellCorrector

In [16]:
# print(df.loc[5, 'Body_01'])
# print('-------------------------------------------------------')
# print(corrector.FixFragment(df.loc[5, 'Body_01']))

In [17]:
def spellCorrectionofText(df):
    corrector = TSpellCorrector()
    assert corrector.LoadLangModel('en.bin')

    correctedTexts = []
    numberOfSpellMistakes = []

    for index in range(len(df['Body_01'])):

        # text = df['Body_01'][index]
        split_text = df['Body_01'][index].split(' ')

        new_text = ""
        numberMistakes = 0

        for word in split_text:
            corrected_word = corrector.FixFragment(word)
            if word != corrected_word:
                numberMistakes += 1
            new_text += corrected_word + " "
            
        numberOfSpellMistakes.append(numberMistakes)
        correctedTexts.append(new_text)

    df['Body_02'] = correctedTexts
    df['Number of spell mistakes'] = numberOfSpellMistakes

spellCorrectionofText(df)


In [18]:
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...,country singer jason aldean who was performing...,0
...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1,shanghai reuters china said it plans to accept...,shanghai reuters china said it plans to accept...,1


# Лемматизация

In [19]:
import sys
!{sys.executable} -m pip install spacy

!{sys.executable} -m spacy download en

     |████████████████████████████████| 12.0 MB 4.8 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [20]:
import spacy

In [21]:
def lemmatizeText(df):

    lemmatizer = spacy.load('en')

    df['Body_03'] = df['Body_02']

    for index in range(len(df['Body_02'])):
        doc = lemmatizer(df['Body_03'][index])
        sentence = " ".join([token.lemma_ for token in doc])

        df['Body_03'][index] = sentence

    df['Body_03'] = df['Body_03'].str.replace("-PRON-", '')
    df['Body_03'] = df['Body_03'].str.replace("s", '')
    df['Body_03'] = df['Body_03'].str.replace("  ", '')

lemmatizeText(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [22]:
print(df['Body_03'][0])
print('-----------------------------------------')
print(df['Body_02'][0])

image copyright getty image unday morning donald trump go off on a twitt tirade againt a member ofown party thi inin t exactly huge new far from the firt time the preident have turnrhetorical cannon onown rank thi time howeverattack be particularly biting and peronaleentially call tenneee enator bob corker the chair of the powerful enate foreign relation committee a coward for not run for re electionay mr corker beg for the preidentendorement whichrefue to givewrongly claim that mr corkerupport of the iranian nuclear agreement beonly political accomplihment unlike ome ofcolleague mr corker free from have to worry aboutimmediate political future didn t holdtongue kip twitter pot by enbobcorker a hame the white houe have become an adult day care center omeone obviouly mihift thi morning enator bob corker enbobcorker october reportthat wan t the end ofthoughthen peak with the new york time and really let the preident havehere be four choice quote from the tenneee enatorinterview with the 

# Удаление стоп-слов

In [23]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
english_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def removeStopWords(df):
    df['Body_04'] = df['Body_03']

    for stop_word in english_stopwords:
        stopword = r"\b" + stop_word + r"\b"
        df['Body_04'] = df['Body_04'].str.replace(stopword, '')

    df['Body_04'] = df['Body_04'].str.replace('  ', ' ')
    df['Body_04'] = df['Body_04'].str.replace('  ', ' ')

removeStopWords(df)

In [25]:
print(df['Body_03'][0])
print('-----------------------------------------')
print(df['Body_04'][0])

image copyright getty image unday morning donald trump go off on a twitt tirade againt a member ofown party thi inin t exactly huge new far from the firt time the preident have turnrhetorical cannon onown rank thi time howeverattack be particularly biting and peronaleentially call tenneee enator bob corker the chair of the powerful enate foreign relation committee a coward for not run for re electionay mr corker beg for the preidentendorement whichrefue to givewrongly claim that mr corkerupport of the iranian nuclear agreement beonly political accomplihment unlike ome ofcolleague mr corker free from have to worry aboutimmediate political future didn t holdtongue kip twitter pot by enbobcorker a hame the white houe have become an adult day care center omeone obviouly mihift thi morning enator bob corker enbobcorker october reportthat wan t the end ofthoughthen peak with the new york time and really let the preident havehere be four choice quote from the tenneee enatorinterview with the 

In [26]:
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...,country singer jason aldean who was performing...,0,country inger jaon aldean who be perform a the...,country inger jaon aldean perform la vega hoot...
...,...,...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0,trump jr be oon to give a minute peech for of ...,trump jr oon give minute peech reader think th...
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1,shanghai reuters china said it plans to accept...,shanghai reuters china said it plans to accept...,1,hanghai reuter china ayplan to accept datum fr...,hanghai reuter china ayplan accept datum overe...


# Тональность и субъективность

In [30]:
from textblob import TextBlob

In [33]:
def polarityOfText(df):

    polarity_text = []

    for text in df['Body_04']:
        analysisPol = TextBlob(text).polarity
        polarity_text.append(analysisPol)

    df['Polarity'] = polarity_text

def subjectivityOfText(df):
    subjectivity_text = []

    for text in df['Body_04']:
        analysisSub = TextBlob(text).subjectivity
        subjectivity_text.append(analysisSub)

    df['Subjectivity'] = subjectivity_text


polarityOfText(df)
subjectivityOfText(df)

In [44]:
from sklearn.preprocessing import MinMaxScaler

In [54]:
df.loc[:,['Polarity', 'Subjectivity']]

,Polarity,Subjectivity
0,0.049941,0.422146
1,0.076455,0.460599
2,0.135501,0.408650
3,0.119898,0.282653
4,0.043561,0.408333
...,...,...
3983,0.800000,0.750000
3984,0.321465,0.516919
3985,0.090909,0.303030
3986,0.095141,0.309675


In [56]:
scaler = MinMaxScaler()
p = scaler.fit_transform(df.loc[:,['Polarity', 'Subjectivity']])
df.loc[:,['Polarity', 'Subjectivity']] = p

In [57]:
df

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04,Polarity,Subjectivity
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...,0.457409,0.422146
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...,0.474333,0.460599
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...,0.512022,0.408650
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...,0.502063,0.282653
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,country singer jason aldean who was performing...,country singer jason aldean who was performing...,0,country inger jaon aldean who be perform a the...,country inger jaon aldean perform la vega hoot...,0.453337,0.408333
...,...,...,...,...,...,...,...,...,...,...,...
3983,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...,0.936170,0.750000
3984,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...,0.630722,0.516919
3985,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0,trump jr be oon to give a minute peech for of ...,trump jr oon give minute peech reader think th...,0.483559,0.303030
3986,https://www.reuters.com/article/us-china-pharm...,China to accept overseas trial data in bid to ...,SHANGHAI (Reuters) - China said it plans to ac...,1,shanghai reuters china said it plans to accept...,shanghai reuters china said it plans to accept...,1,hanghai reuter china ayplan to accept datum fr...,hanghai reuter china ayplan accept datum overe...,0.486260,0.309675


# Чистка от неработающих

In [58]:
index_to_remove = [4, 18, 19, 20, 21, 24, 26, 27, 28, 30, 31, 35, 42, 43, 45, 51, 61, 66, 68, 70, 78, 79, 80, 82, 85, 96, 101, 102, 103,
                   104, 108, 113, 115, 118, 122, 123, 124, 128, 129, 131, 132, 133, 135, 136, 137, 140, 143, 145, 146, 153, 156, 160, 164,
                   165, 170, 171, 176, 179, 180, 182, 183, 189, 191, 192, 193, 195, 201, 202, 203, 204, 208, 209, 210, 212, 213, 215, 216,
                   218, 219, 220, 221, 222, 223, 226, 227, 230, 232, 235, 237, 239, 240, 243, 245, 251, 254, 255, 264, 265, 272, 274, 
                   275, 277, 280, 281, 282, 283, 286, 287, 290, 292, 294, 295, 300, 301, 306, 308, 311, 
                   315, 319, 320, 321, 323, 326, 327, 328, 331, 334, 335, 337, 338, 340, 342, 346, 347, 349, 352, 353, 354, 355, 
                   361, 364, 365, 366, 367, 369, 370, 371, 373, 374, 376, 378, 380, 381, 382, 383, 384, 385, 387, 388, 389, 391, 
                   392, 393, 394, 395, 396, 398, 399, 402, 403, 405, 406, 408, 409, 410, 415, 418, 420, 421, 422, 432, 433, 436, 
                   439, 444, 450, 455, 456, 457, 459, 461, 462, 466, 467, 468, 470, 471, 472, 473, 475, 477, 480, 481, 486, 489, 
                   492, 494, 498, 499, 502, 504, 506, 508, 510, 513, 514, 516, 520, 522, 523, 524, 525, 527, 529, 530, 535, 538, 
                   542, 554, 555, 556, 557, 558, 559, 561, 563, 564, 565, 566, 567, 569, 572, 577, 581, 584, 586, 588, 590, 596, 
                   598, 603, 604, 607, 608, 609, 611, 614, 617, 620, 622, 623, 626, 627, 631, 635, 636, 637, 638, 642, 644, 646, 
                   649, 650, 651, 653, 654, 655, 657, 658, 661, 662, 667, 668, 672, 674, 676, 677, 680, 681, 685, 687, 694, 697, 
                   698, 700, 701, 710, 712, 714, 716, 717, 720, 721, 723, 731, 733, 734, 735, 737, 739, 742, 745, 746, 747, 750, 
                   753, 754, 756, 761, 763, 773, 774, 783, 784, 786, 787, 791, 792, 795, 796, 802, 807, 810, 812, 817, 818, 823, 
                   824, 828, 835, 842, 843, 844, 846, 851, 852, 854, 857, 859, 863, 864, 869, 875, 876, 877, 879, 881, 882, 883, 
                   884, 885, 888, 891, 892, 893, 895, 896, 902, 903, 905, 909, 918, 922, 923, 924, 926, 927, 929, 932, 934, 935, 
                   936, 937, 939, 940, 941, 942, 943, 946, 948, 950, 951, 952, 954, 956, 957, 960, 963, 965, 966, 967, 968, 973, 
                   974, 981, 983, 984, 986, 988, 989, 990, 991, 993, 994, 995, 998, 1001, 1003, 1009, 1010, 1014, 1015, 1016, 
                   1021, 1023, 1025, 1027, 1030, 1034, 1039, 1040, 1043, 1045, 1048, 1051, 1052, 1054, 1056, 1057, 1062, 
                   1070, 1071, 1072, 1074, 1076, 1077, 1078, 1079, 1080, 1081, 1083, 1085, 1086, 1087, 1089, 1092, 1093, 
                   1094, 1095, 1096, 1103, 1104, 1105, 1107, 1112, 1114, 1116, 1117, 1120, 1123, 1124, 1126, 1127, 1130, 
                   1133, 1134, 1135, 1138, 1140, 1141, 1142, 1143, 1144, 1145, 1149, 1158, 1159, 1161, 1162, 1168, 1169, 
                   1170, 1174, 1178, 1184, 1185, 1186, 1187, 1188, 1190, 1192, 1198, 1202, 1203, 1210, 1211, 1213, 1214, 
                   1215, 1217, 1219, 1222, 1224, 1225, 1228, 1229, 1231, 1235, 1236, 1239, 1240, 1242, 1247, 1248, 1249, 
                   1250, 1251, 1252, 1257, 1268, 1269, 1270, 1271, 1275, 1280, 1283, 1291, 1293, 1297, 1300, 1302, 1305, 
                   1307, 1308, 1309, 1310, 1312, 1313, 1317, 1318, 1321, 1322, 1327, 1329, 1331, 1333, 1340, 1344, 1347, 
                   1348, 1353, 1354, 1356, 1357, 1360, 1362, 1374, 1376, 1377, 1378, 1379, 1382, 1388, 1389, 1393, 1396, 
                   1399, 1401, 1402, 1404, 1409, 1411, 1412, 1413, 1418, 1419, 1423, 1427, 1428, 1432, 1433, 1434, 1438, 
                   1440, 1441, 1442, 1444, 1445, 1446, 1448, 1449, 1451, 1452, 1455, 1458, 1459, 1464, 1465, 1466, 1467, 
                   1469, 1471, 1473, 1477, 1479, 1482, 1484, 1486, 1487, 1489, 1490, 1491, 1492, 1501, 1502, 1504, 1508, 
                   1511, 1513, 1514, 1515, 1516, 1517, 1519, 1520, 1523, 1524, 1527, 1528, 1529, 1531, 1532, 1534, 1535, 
                   1538, 1540, 1541, 1543, 1545, 1546, 1550, 1551, 1553, 1556, 1560, 1563, 1565, 1568, 1569, 1574, 1575, 
                   1579, 1582, 1585, 1586, 1592, 1593, 1598, 1600, 1603, 1606, 1607, 1609, 1611, 1613, 1617, 1618, 1622, 
                   1625, 1628, 1629, 1632, 1633, 1641, 1642, 1648, 1649, 1650, 1652, 1653, 1654, 1656, 1658, 1659, 1662, 
                   1663, 1668, 1671, 1674, 1677, 1680, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1696, 1697, 1699, 1701, 
                   1703, 1704, 1705, 1706, 1709, 1710, 1720, 1721, 1722, 1725, 1730, 1733, 1743, 1747, 1751, 1753, 1754, 
                   1755, 1756, 1762, 1763, 1765, 1767, 1769, 1770, 1771, 1777, 1778, 1780, 1782, 1785, 1786, 1787, 1790, 
                   1791, 1793, 1798, 1800, 1802, 1805, 1806, 1808, 1810, 1812, 1814, 1815, 1818, 1819, 1821, 1825, 1828, 
                   1829, 1831, 1834, 1836, 1838, 1841, 1842, 1843, 1844, 1845, 1847, 1849, 1850, 1851, 1853, 1854, 1855, 
                   1856, 1857, 1858, 1860, 1866, 1867, 1868, 1874, 1875, 1878, 1879, 1882, 1884, 1885, 1888, 1896, 1897, 
                   1903, 1904, 1912, 1913, 1915, 1918, 1920, 1923, 1924, 1925, 1926, 1928, 1931, 1932, 1935, 1937, 1938, 
                   1939, 1942, 1946, 1949, 1950, 1956, 1960, 1962, 1965, 1966, 1967, 1970, 1974, 1975, 1982, 1983, 1987, 
                   1989, 1990, 1996, 1997, 1998, 2001, 2002, 2010, 2014, 2015, 2016, 2022, 2025, 2026, 2028, 2032, 2033, 
                   2034, 2041, 2045, 2046, 2048, 2052, 2055, 2056, 2061, 2062, 2063, 2064, 2066, 2067, 2070, 2073, 2075, 
                   2076, 2077, 2078, 2079, 2084, 2087, 2089, 2091, 2093, 2094, 2096, 2097, 2102, 2109, 2111, 2115, 2119, 
                   2120, 2121, 2122, 2124, 2127, 2130, 2131, 2133, 2134, 2136, 2138, 2139, 2142, 2144, 2151, 2152, 2153, 
                   2155, 2160, 2163, 2170, 2173, 2174, 2175, 2178, 2182, 2186, 2187, 2189, 2193, 2194, 2196, 2199, 2200, 
                   2201, 2202, 2204, 2210, 2213, 2214, 2216, 2220, 2227, 2228, 2229, 2231, 2233, 2238, 2240, 2243, 2245, 
                   2249, 2251, 2253, 2255, 2256, 2257, 2258, 2260, 2267, 2268, 2273, 2277, 2280, 2281, 2282, 2285, 2286, 
                   2287, 2288, 2290, 2291, 2294, 2300, 2301, 2303, 2304, 2312, 2316, 2317, 2321, 2322, 2323, 2327, 2330, 
                   2331, 2333, 2337, 2339, 2349, 2351, 2355, 2357, 2361, 2366, 2367, 2368, 2370, 2374, 2375, 2376, 2379, 
                   2380, 2383, 2384, 2389, 2390, 2393, 2395, 2396, 2397, 2398, 2402, 2406, 2408, 2413, 2414, 2419, 2421, 
                   2423, 2424, 2428, 2429, 2430, 2432, 2434, 2436, 2438, 2444, 2447, 2451, 2456, 2469, 2480, 2484, 2486, 
                   2487, 2489, 2490, 2491, 2497, 2502, 2509, 2510, 2511, 2513, 2516, 2525, 2526, 2531, 2538, 2540, 2541, 
                   2546, 2548, 2550, 2552, 2553, 2554, 2559, 2561, 2563, 2564, 2565, 2569, 2572, 2574, 2575, 2577, 2579, 
                   2580, 2586, 2588, 2589, 2591, 2595, 2599, 2600, 2602, 2603, 2604, 2605, 2606, 2608, 2609, 2610, 2614, 
                   2618, 2619, 2620, 2622, 2623, 2626, 2627, 2628, 2630, 2637, 2639, 2647, 2651, 2652, 2659, 2660, 2661, 
                   2665, 2666, 2667, 2670, 2671, 2677, 2678, 2681, 2682, 2683, 2686, 2689, 2690, 2691, 2694, 2695, 2697, 
                   2699, 2701, 2703, 2704, 2705, 2707, 2709, 2710, 2712, 2714, 2716, 2718, 2721, 2723, 2725, 2734, 2738, 
                   2742, 2749, 2752, 2754, 2757, 2759, 2760, 2761, 2762, 2766, 2767, 2772, 2779, 2781, 2782, 2784, 2785, 
                   2787, 2794, 2796, 2797, 2801, 2803, 2804, 2805, 2806, 2808, 2820, 2822, 2824, 2827, 2829, 2830, 2835, 
                   2840, 2841, 2842, 2843, 2844, 2849, 2854, 2857, 2859, 2862, 2863, 2865, 2867, 2868, 2870, 2871, 2874, 
                   2875, 2876, 2880, 2881, 2886, 2891, 2894, 2899, 2900, 2902, 2904, 2906, 2907, 2909, 2919, 2921, 2922, 
                   2925, 2928, 2929, 2930, 2933, 2934, 2937, 2938, 2940, 2941, 2943, 2944, 2953, 2954, 2957, 2959, 2968, 
                   2974, 2976, 2980, 2981, 2983, 2984, 2985, 2986, 2987, 2988, 2989, 2995, 3001, 3010, 3015, 3016, 3017, 
                   3021, 3022, 3023, 3025, 3026, 3032, 3038, 3042, 3048, 3052, 3055, 3057, 3058, 3059, 3061, 3063, 3066, 
                   3069, 3070, 3071, 3075, 3076, 3077, 3078, 3080, 3082, 3086, 3087, 3090, 3091, 3092, 3094, 3096, 3104, 
                   3106, 3107, 3109, 3111, 3112, 3114, 3115, 3117, 3119, 3121, 3124, 3127, 3128, 3130, 3132, 3134, 3141, 
                   3149, 3151, 3156, 3157, 3161, 3168, 3170, 3171, 3173, 3176, 3179, 3181, 3183, 3186, 3188, 3189, 3197, 
                   3198, 3200, 3201, 3202, 3207, 3211, 3214, 3215, 3216, 3217, 3220, 3224, 3226, 3232, 3234, 3238, 3243, 
                   3246, 3248, 3250, 3252, 3253, 3261, 3264, 3273, 3276, 3277, 3278, 3283, 3284, 3286, 3288, 3290, 3296, 
                   3297, 3300, 3301, 3304, 3305, 3307, 3314, 3316, 3321, 3326, 3327, 3329, 3330, 3332, 3335, 3336, 3340, 
                   3341, 3342, 3344, 3349, 3351, 3352, 3353, 3355, 3357, 3358, 3360, 3363, 3364, 3365, 3366, 3367, 3370, 
                   3373, 3374, 3376, 3377, 3378, 3379, 3380, 3381, 3382, 3383, 3388, 3390, 3393, 3397, 3398, 3399, 3410, 
                   3412, 3414, 3416, 3417, 3418, 3420, 3423, 3424, 3425, 3427, 3428, 3434, 3436, 3437, 3439, 3441, 3444, 
                   3448, 3449, 3452, 3453, 3462, 3466, 3467, 3468, 3470, 3471, 3472, 3473, 3478, 3480, 3483, 3485, 3486, 
                   3487, 3489, 3490, 3491, 3498, 3499, 3501, 3505, 3506, 3513, 3514, 3516, 3517, 3518, 3522, 3528, 3529, 
                   3530, 3535, 3539, 3542, 3544, 3547, 3549, 3550, 3552, 3554, 3555, 3556, 3557, 3562, 3563, 3566, 3569, 
                   3574, 3575, 3579, 3581, 3582, 3584, 3585, 3587, 3588, 3590, 3592, 3595, 3596, 3597, 3598, 3600, 3601, 
                   3603, 3606, 3610, 3611, 3612, 3613, 3614, 3618, 3620, 3622, 3629, 3630, 3631, 3636, 3639, 3642, 3651, 
                   3661, 3663, 3664, 3668, 3670, 3672, 3673, 3675, 3676, 3680, 3682, 3683, 3684, 3685, 3688, 3692, 3699, 
                   3701, 3710, 3713, 3716, 3717, 3721, 3722, 3723, 3730, 3731, 3735, 3736, 3737, 3738, 3739, 3743, 3745, 
                   3746, 3749, 3751, 3752, 3755, 3757, 3758, 3760, 3761, 3762, 3765, 3771, 3775, 3776, 3779, 3785, 3786, 
                   3788, 3789, 3791, 3793, 3794, 3795, 3797, 3798, 3799, 3803, 3810, 3817, 3818, 3820, 3822, 3823, 3825, 
                   3826, 3828, 3830, 3832, 3833, 3834, 3835, 3836, 3837, 3838, 3841, 3844, 3846, 3847, 3848, 3849, 3851, 
                   3852, 3853, 3856, 3857, 3859, 3860, 3869, 3872, 3873, 3876, 3877, 3884, 3888, 3889, 3893, 3899, 3901, 
                   3902, 3904, 3905, 3906, 3907, 3911, 3912, 3915, 3916, 3920, 3921, 3922, 3926, 3931, 3933, 3942, 3943, 
                   3944, 3946, 3948, 3949, 3950, 3951, 3962, 3966, 3967, 3972, 3973, 3975, 3979, 3981, 3982, 3986]


In [59]:
df_clean = df.drop(index_to_remove).reset_index()

In [60]:
df_clean = df_clean.drop(columns=['index'], axis = 1)

In [61]:
df_clean

,URLs,Headline,Body,Label,Body_01,Body_02,Number of spell mistakes,Body_03,Body_04,Polarity,Subjectivity
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,image copyright getty imageson sunday morning ...,image copyright getty images sunday morning do...,4,image copyright getty image unday morning dona...,image copyright getty image unday morning dona...,0.457409,0.422146
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,london reuters last flag flying a comedy dram...,london reuters last flag flying a comedy dram...,0,london reuter lat flag fly a comedy drama abou...,london reuter lat flag fly comedy drama vietna...,0.474333,0.460599
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,the feud broke into public view last week when...,the feud broke into public view last week when...,3,the feud break into public view lat week when ...,feud break public view lat week mr corker ay ...,0.512022,0.408650
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,mexico city reuters egypt s cheiron holdings l...,mexico city reuters egypt s chevron holdings l...,1,mexico city reuter egyptchevron holding limite...,mexico city reuter egyptchevron holding limite...,0.502063,0.282653
4,http://beforeitsnews.com/sports/2017/09/jetnat...,JetNation FanDuel League; Week 4,JetNation FanDuel League; Week 4\n% of readers...,0,jetnation fanduel league week of readers think...,detention fanduel league week of readers think...,3,detention fanduel league week of reader think ...,detention fanduel league week reader think thi...,0.425532,0.337500
...,...,...,...,...,...,...,...,...,...,...,...
2371,https://www.activistpost.com/2017/09/false-arr...,35 False Matches and 1 Erroneous Arrest As Pol...,By Nicholas West\nAll things biometric are swe...,0,by nicholas westall things biometric are sweep...,by nicholas westall things biometric are sweep...,1,by nichola wetall thing biometric be weep acro...,nichola wetall thing biometric weep acro worl...,0.483309,0.308180
2372,http://beforeitsnews.com/u-s-politics/2017/10/...,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,vietnam is in great danger you must publish an...,vietnam is in great danger you must publish an...,0,vietnam be in great dangermut publih and tella...,vietnam great dangermut publih tellarmy govern...,0.936170,0.750000
2373,http://beforeitsnews.com/sports/2017/09/trends...,Trends to Watch,Trends to Watch\n% of readers think this story...,0,trends to watch of readers think this story is...,trends to watch of readers think this story is...,8,trend to watch of reader think thi tory be fac...,trend watch reader think thi tory fact addtwo ...,0.630722,0.516919
2374,http://beforeitsnews.com/u-s-politics/2017/10/...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr is soon to give a minute speech for ...,trump jr is soon to give a minute speech for ...,0,trump jr be oon to give a minute peech for of ...,trump jr oon give minute peech reader think th...,0.483559,0.303030


In [62]:
df_clean.to_csv("clean_text_analisis.csv", index=False)
files.download("clean_text_analisis.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Векторизация

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
# tfidf = TfidfVectorizer(encoding='utf-8',
#                         ngram_range=(1,3),
#                         stop_words=None,
#                         lowercase=False,
#                         norm='l2',
#                         sublinear_tf=True)
                        
# features = tfidf.fit_transform(df['Body_04']).toarray() # fit()-обучение transfrom()-применение
# feature_names = tfidf.get_feature_names()

# df_features = pd.DataFrame(features, columns=feature_names)

# # features_test = tfidf.transform(X_test).toarray()
# # labels_test = y_test
# # print(features_test.shape)

In [29]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# # text_parsed = cv.fit_transform(text_parsed)